<a href="https://colab.research.google.com/github/Jakelinecs/Tareas-Machine-Learning/blob/main/N32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import reuters # ¡Cambiado a Reuters!
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, GRU, LSTM, Dense
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical # Importado para One-Hot Encoding
import time

# --- Configuración de Parámetros ---

# Límite del vocabulario (solo se consideran las 'max_words' más frecuentes)
MAX_WORDS = 10000
# Longitud máxima de la secuencia
MAX_LEN = 100
# Dimensiones de la capa de embedding
EMBEDDING_DIM = 32
# Número de nodos recurrentes (unidades)
RNN_UNITS = 32
# Épocas de entrenamiento (se mantiene bajo para una ejecución rápida)
EPOCHS = 3
# Tamaño del lote
BATCH_SIZE = 128
# Número de clases para Reuters (46 temas)
NUM_CLASSES = 46

# --- Carga y Preprocessing de Datos ---

def load_and_preprocess_data():
    """Carga el conjunto de datos Reuters y lo preprocesa para clasificación multiclase."""
    print("Cargando datos de Reuters Newswire...")
    # Carga de datos de entrenamiento y prueba
    (x_train, y_train), (x_test, y_test) = reuters.load_data(num_words=MAX_WORDS)

    print(f"Número de secuencias de entrenamiento: {len(x_train)}")
    print(f"Número de secuencias de prueba: {len(x_test)}")
    print(f"Número de clases: {NUM_CLASSES}")

    # Relleno (padding) de secuencias
    print("Rellenando secuencias...")
    x_train = pad_sequences(x_train, maxlen=MAX_LEN, padding='post', truncating='post')
    x_test = pad_sequences(x_test, maxlen=MAX_LEN, padding='post', truncating='post')

    # CONVERSIÓN A ONE-HOT ENCODING (Necesario para 'categorical_crossentropy')
    y_train = to_categorical(y_train, num_classes=NUM_CLASSES)
    y_test = to_categorical(y_test, num_classes=NUM_CLASSES)

    return x_train, y_train, x_test, y_test

# --- Definición del Modelo y Entrenamiento ---

def build_and_train_model(rnn_layer_type, rnn_units, x_train, y_train, x_test, y_test):
    """
    Construye, compila y entrena un modelo Keras con la capa RNN especificada.
    Adaptado para clasificación multiclase (Reuters).
    """
    print(f"\n--- Entrenando modelo con capa: {rnn_layer_type.__name__} ---")

    # 1. Definición del Modelo
    model = Sequential()

    # Capa de Embedding
    model.add(Embedding(MAX_WORDS, EMBEDDING_DIM, input_length=MAX_LEN))

    # 2. Capa Recurrente
    model.add(rnn_layer_type(rnn_units))

    # 3. Capa de Salida (Clasificación Multiclase: 46 temas)
    # Usando NUM_CLASSES y activación 'softmax'
    model.add(Dense(NUM_CLASSES, activation='softmax'))

    # 4. Compilación
    model.compile(
        optimizer='adam',
        # Usando 'categorical_crossentropy' para clasificación multiclase
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # 5. Entrenamiento
    start_time = time.time()
    history = model.fit(
        x_train, y_train,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        validation_data=(x_test, y_test),
        verbose=1
    )
    end_time = time.time()

    # 6. Evaluación
    loss, acc = model.evaluate(x_test, y_test, verbose=0)

    print(f"Tiempo de entrenamiento: {end_time - start_time:.2f} segundos")
    print(f"Precisión (Accuracy) en datos de prueba: {acc:.4f}")

    # Print training and validation accuracy per epoch
    print("\nTraining and Validation Accuracy per Epoch:")
    for epoch in range(EPOCHS):
        print(f"Epoch {epoch+1}: Train Acc = {history.history['accuracy'][epoch]:.4f}, Val Acc = {history.history['val_accuracy'][epoch]:.4f}")


    return model, history, acc

# --- Función Principal de Comparación ---

def rnn_comparison():
    """Ejecuta la comparación entre SimpleRNN, GRU y LSTM con el dataset Reuters."""

    x_train, y_train, x_test, y_test = load_and_preprocess_data()

    results = {}

    # Lista de capas RNN a comparar (SimpleRNN, GRU, LSTM)
    rnn_layers = [SimpleRNN, GRU, LSTM]

    for rnn_layer in rnn_layers:
        _, history, acc = build_and_train_model(
            rnn_layer,
            RNN_UNITS,
            x_train, y_train, x_test, y_test
        )
        results[rnn_layer.__name__] = acc

    print("\n" + "="*50)
    print("RESUMEN DE PRECISIÓN EN DATOS DE PRUEBA (Reuters)")
    print("="*50)
    for name, accuracy in sorted(results.items(), key=lambda item: item[1], reverse=True):
        print(f"| {name:<12}: {accuracy:.4f} |")
    print("="*50)

    # Demostración adicional de ConvLSTM2D (no comparable directamente)
    print("\n--- Demostración de ConvLSTM2D ---")
    print("ConvLSTM2D requiere un tensor 5D (muestras, tiempo, filas, columnas, canales).")
    print("No es directamente aplicable a este dataset de texto plano.")


if __name__ == '__main__':
    # Configurar el nivel de log de TensorFlow para no mostrar mensajes de advertencia excesivos
    tf.get_logger().setLevel('ERROR')
    rnn_comparison()

Cargando datos de Reuters Newswire...
Número de secuencias de entrenamiento: 8982
Número de secuencias de prueba: 2246
Número de clases: 46
Rellenando secuencias...

--- Entrenando modelo con capa: SimpleRNN ---
Epoch 1/3


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


71/71 ━━━━━━━━━━━━━━━━━━━━ 6s 40ms/step - accuracy: 0.3138 - loss: 3.1002 - val_accuracy: 0.3620 - val_loss: 2.3975
Epoch 2/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 3s 36ms/step - accuracy: 0.3511 - loss: 2.3741 - val_accuracy: 0.3687 - val_loss: 2.3438
Epoch 3/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 2s 35ms/step - accuracy: 0.3724 - loss: 2.3429 - val_accuracy: 0.3713 - val_loss: 2.4537
Tiempo de entrenamiento: 10.78 segundos
Precisión (Accuracy) en datos de prueba: 0.3713

Training and Validation Accuracy per Epoch:
Epoch 1: Train Acc = 0.3370, Val Acc = 0.3620
Epoch 2: Train Acc = 0.3614, Val Acc = 0.3687
Epoch 3: Train Acc = 0.3844, Val Acc = 0.3713

--- Entrenando modelo con capa: GRU ---
Epoch 1/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 10s 91ms/step - accuracy: 0.3291 - loss: 3.3507 - val_accuracy: 0.3620 - val_loss: 2.3882
Epoch 2/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 7s 97ms/step - accuracy: 0.3657 - loss: 2.3193 - val_accuracy: 0.3633 - val_loss: 2.3030
Epoch 3/3
71/71 ━━━━━━━━━━━━━━━━━━━━ 6s 86ms/step - accuracy: 0.363